In [42]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [43]:
neighborhoods = pd.read_csv("ClusterCanada/canada.csv", sep=';')

In [44]:
neighborhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194350
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160490
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188710
3,M1G,Scarborough,Woburn,43.770992,-79.216910
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [45]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

neighborhoods.shape

The dataframe has 11 boroughs and 103 neighborhoods.


(103, 5)

#### Use geopy library to get the latitude and longitude values of Canada.

In [46]:
#import geocoder
# import geocoder

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

latitude = 43.3914
longitude = -79.233




In [47]:
# create map using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [48]:
east_toronto_data = neighborhoods[neighborhoods['Borough'] == 'East Toronto'].reset_index(drop=True)
east_toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.29303
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.35218
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.31557
3,M4M,East Toronto,Studio District,43.659526,-79.34092
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.32155


#### Foursquare Access

In [50]:
CLIENT_ID = '4XEOYQPW1HE1MH1ILMRPLKH1UZTDTBL40GNIXC532ODLNZRL' # your Foursquare ID
CLIENT_SECRET = 'WSBY442O4IE4REFXVN51V311NMMMHB4HH5KYRBRCPH0QALNE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4XEOYQPW1HE1MH1ILMRPLKH1UZTDTBL40GNIXC532ODLNZRL
CLIENT_SECRET:WSBY442O4IE4REFXVN51V311NMMMHB4HH5KYRBRCPH0QALNE


In [51]:
east_toronto_data.loc[0, 'Neighbourhood']

'The Beaches'

In [53]:
neighborhood_latitude = east_toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = east_toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = east_toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.29303.


### Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.

In [55]:
# type your answer here
radius=5
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude,radius, LIMIT)


# display URL
url


'https://api.foursquare.com/v2/venues/explore?&client_id=4XEOYQPW1HE1MH1ILMRPLKH1UZTDTBL40GNIXC532ODLNZRL&client_secret=WSBY442O4IE4REFXVN51V311NMMMHB4HH5KYRBRCPH0QALNE&v=20180605&ll=43.67635739999999,-79.29303&radius=5&limit=100'

#### Result from GET

In [56]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cd7eebfdb04f559d639226b'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 43.67640240004499,
    'lng': -79.2929678971449},
   'sw': {'lat': 43.676312399954995, 'lng': -79.2930921028551}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [58]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [61]:
east_toronto_venues = getNearbyVenues(names=east_toronto_data['Neighbourhood'],
                                   latitudes=east_toronto_data['Latitude'],
                                   longitudes=east_toronto_data['Longitude']
                                  )

The Beaches
Riverdale, The Danforth West
India Bazaar, The Beaches West
Studio District
Business Reply Mail Processing Centre 969 Eastern


In [65]:
print(east_toronto_venues.shape)
east_toronto_venues.head()

(124, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.29303,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.29303,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.29303,St-Denis Studios Inc.,43.675031,-79.288022,Music Venue
3,The Beaches,43.676357,-79.29303,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"Riverdale, The Danforth West",43.679557,-79.35218,Pantheon,43.677621,-79.351434,Greek Restaurant


In [66]:
east_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"India Bazaar, The Beaches West",22,22,22,22,22,22
"Riverdale, The Danforth West",44,44,44,44,44,44
Studio District,37,37,37,37,37,37
The Beaches,4,4,4,4,4,4


In [67]:
print('There are {} uniques categories.'.format(len(east_toronto_venues['Venue Category'].unique())))

There are 68 uniques categories.


In [71]:
# one hot encoding
east_toronto_onehot = pd.get_dummies(east_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
east_toronto_onehot['Neighborhood'] = east_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [east_toronto_onehot.columns[-1]] + list(east_toronto_onehot.columns[:-1])
east_toronto_onehot = east_toronto_onehot[fixed_columns]

east_toronto_onehot.head()


,Yoga Studio,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Music Store,Music Venue,Neighborhood,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Trail
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Riverdale, The Danforth West",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [72]:
east_toronto_onehot.shape

(124, 68)

In [74]:
east_toronto_grouped = east_toronto_onehot.groupby('Neighborhood').mean().reset_index()
east_toronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Music Store,Music Venue,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Trail
0,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.00,0.058824,0.000000,0.058824,0.000000,0.058824,0.058824,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"India Bazaar, The Beaches West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.00,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.045455,0.000000,0.00,0.090909,0.045455,0.045455,0.045455,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000
2,"Riverdale, The Danforth West",0.022727,0.022727,0.000000,0.022727,0.000000,0.000000,0.045455,0.022727,0.022727,0.000000,0.000000,0.022727,0.022727,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.022727,0.000000,0.022727,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.045455,0.000000,0.000000,0.000000,0.181818,0.022727,0.000000,0.000000,0.00,0.068182,0.022727,0.045455,0.022727,0.022727,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.022727,0.022727,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.022727,0.022727,0.000000,0.000000,0.022727,0.022727
3,Studio District,0.027027,0.054054,0.000000,0.054054,0.027027,0.027027,0.027027,0.027027,0.000000,0.000000,0.000000,0.108108,0.000000,0.027027,0.027027,0.027027,0.081081,0.027027,0.000000,0.027027,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.027027,0.00,0.027027,0.000000,0.054054,0.000000,0.000000,0.027027,0.000000,0.000000,0.027027,0.000000,0.027027,0.00,0.027027,0.000000,0.027027,0.000000,0.000000,0.000000,0.027027,0.027027,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000
4,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,

In [75]:
east_toronto_grouped.shape

(5, 68)

In [76]:
num_top_venues = 5

for hood in east_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = east_toronto_grouped[east_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2          Skate Park  0.06
3       Garden Center  0.06
4              Garden  0.06


----India Bazaar, The Beaches West----
            venue  freq
0            Park  0.09
1  Sandwich Place  0.09
2     Pizza Place  0.05
3     Coffee Shop  0.05
4  Ice Cream Shop  0.05


----Riverdale, The Danforth West----
                    venue  freq
0        Greek Restaurant  0.18
1             Coffee Shop  0.09
2          Ice Cream Shop  0.07
3      Italian Restaurant  0.05
4  Furniture / Home Store  0.05


----Studio District----
                 venue  freq
0                 Café  0.11
1          Coffee Shop  0.08
2               Bakery  0.05
3  American Restaurant  0.05
4   Italian Restaurant  0.05


----The Beaches----
                       venue  freq
0          Health Food Store  0.25
1                        Pub  0.25
2                Music Venue  0

In [77]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = east_toronto_grouped['Neighborhood']

for ind in np.arange(east_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(east_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Farmers Market,Comic Shop,Burrito Place,Park,Pizza Place,Recording Studio,Restaurant,Brewery
1,"India Bazaar, The Beaches West",Park,Sandwich Place,Liquor Store,Food & Drink Shop,Sushi Restaurant,Gym,Coffee Shop,Ice Cream Shop,Italian Restaurant,Light Rail Station
2,"Riverdale, The Danforth West",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Trail,Diner,Indian Restaurant,Grocery Store
3,Studio District,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Ice Cream Shop,Gym / Fitness Center,Fish Market,Coworking Space,Convenience Store
4,The Beaches,Music Venue,Pub,Health Food Store,Trail,Dessert Shop,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Diner


### Cluster Neighborhoods

In [85]:
# set number of clusters
kclusters = 5

east_toronto_grouped_clustering = east_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(east_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 4, 1, 0], dtype=int32)

In [92]:
east_toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.29303
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.35218
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.31557
3,M4M,East Toronto,Studio District,43.659526,-79.34092
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.32155


In [96]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


east_toronto_merged = east_toronto_data 

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#east_toronto_merged = east_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

east_toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.29303
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.35218
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.31557
3,M4M,East Toronto,Studio District,43.659526,-79.34092
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.32155
